# SQL
[Learning SQL](http://www.r-5.org/files/books/computers/languages/sql/mysql/Alan_Beaulieu-Learning_SQL-EN.pdf)

In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to a DB
conn = sqlite3.connect('data/mtcars.sqlite')

In [3]:
# List tables
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

,name
0,results


In [4]:
# Display table
df = pd.read_sql_query("SELECT * from results", conn)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [5]:
# Simple Query
pd.read_sql_query(
"""
SELECT  
    mpg, 
    displacement,
    name
FROM results
--- this is a SQL code comment
WHERE year > 75
    and acceleration < 18
""", 
    con=conn
)

,mpg,displacement,name
0,28.0,107.0,fiat 131
1,25.0,116.0,opel 1900
2,25.0,140.0,capri ii
3,26.0,98.0,dodge colt
4,27.0,101.0,renault 12tl
...,...,...,...
165,32.0,144.0,toyota celica gt
166,36.0,135.0,dodge charger 2.2
167,27.0,151.0,chevrolet camaro
168,27.0,140.0,ford mustang gl


In [6]:
# Group by
pd.read_sql(
"""
SELECT  
    AVG(mpg) as avg_mpg,
    SUM(mpg) as sum_mpg,
    AVG(horsepower) as hp,
    AVG(acceleration)
FROM results
WHERE year > 75
GROUP BY cylinders
ORDER BY avg_mpg DESC
""", 
    con=conn
)

,avg_mpg,sum_mpg,hp,AVG(acceleration)
0,31.446565,4119.5,74.809160,16.551145
1,27.366667,82.1,82.333333,18.633333
2,22.600000,45.2,105.000000,13.000000
3,21.440000,964.8,103.977778,16.224444
4,17.270588,587.2,140.382353,14.144118


In [14]:
# Select new DB
conn = sqlite3.connect('data/demo.db3')
#pd.read_sql("SELECT * From rch LIMIT 3", con=conn)
pd.read_sql("SELECT * From sub LIMIT 3", con=conn)

,SUB,YR,MO,PRECIPmm,SNOMELTmm,PETmm,ETmm,SWmm,PERCmm,SURQmm,...,SEDPkg_ha,LAT_Q_mm,LATNO3kg_h,GWNO3kg_ha,CHOLAmic_L,CBODU_mg_L,DOXQ_mg_L,TNO3kg_ha,QTILEmm,TVAPkg_ha
0,1,1981,1,35.601990,0.0,3.720744,0.249642,10.798594,0.0,0.000024,...,4.759284e-08,0.007812,0.000000,1.562287e-07,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1981,1,108.606071,0.0,3.450408,0.457206,56.325005,0.0,0.000017,...,3.447771e-08,0.033931,0.000016,3.828798e-04,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1981,1,149.308365,0.0,10.566324,6.027106,71.002098,0.0,0.000024,...,3.778229e-08,0.007915,0.000004,7.582882e-05,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Join
df = pd.read_sql(
"""
SELECT 
    RCH, 
    rch.YR, 
    rch.MO, 
    FLOW_INcms, 
    FLOW_OUTcms, 
    PRECIPmm, 
    PETmm
FROM rch 
LEFT JOIN sub
    ON rch.RCH = sub.SUB 
    and rch.YR=sub.YR and rch.MO=sub.MO
WHERE
    rch.YR > 1984
""", con=conn)

print(len(df))
df.head()

7176


,RCH,YR,MO,FLOW_INcms,FLOW_OUTcms,PRECIPmm,PETmm
0,1,1985,1,65.014122,64.851318,46.502602,6.640418
1,2,1985,1,44.200371,44.159149,103.605797,3.325713
2,3,1985,1,2.623852,2.441326,119.506691,21.123943
3,4,1985,1,20.668079,20.552505,103.605797,15.438934
4,5,1985,1,354.287933,348.826935,146.608200,75.672722


In [15]:
# Nested Query
df = pd.read_sql(
"""
SELECT 
    RCH, 
    Quarter, 
    AVG(FLOW_OUTcms) as Runoff
--- This is a subquery
FROM (
    SELECT RCH, YR, 
    CASE 
        WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
        WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
        WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
        ELSE 'DJF' 
    END Quarter,
    FLOW_OUTcms
    from rch
)
GROUP BY RCH, Quarter
""", con=conn)

print(len(df))
df.head()

92


,RCH,Quarter,Runoff
0,1,DJF,99.204991
1,1,JJA,1405.262298
2,1,MAM,559.746932
3,1,SON,454.737985
4,2,DJF,56.328539


In [9]:
# Making a new table 
df.to_sql(
    name="test_output",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)

In [10]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

,name
0,rch
1,sub
2,sqlite_sequence
3,test_output


In [11]:
# Delete table
#df = pd.read_sql("DROP TABLE test_output", con=conn)

In [12]:
# Making a new DB
#conn = sqlite3.connect('test.sqlite')